# Extração de dados do Github

Pesquisaremos por iniciativas/projetos que utilizam Dados Abertos Governamentais através da [API do Github](https://developer.github.com/v3/)

A partir das apontamentos feitos na qualificação desse projeto de pesquisa, entendemos que caracterizar toda a comunidade que utiliza dados abertos governamentais é um escopo muito abrangente e de difícil validação.
A ideia é reduzir esse escopo para assim poder avaliar melhor os seus resultados, por exemplo verificar se os projetos referências nessas áreas aparecem nos repositórios extraídos no Github.

Vamos testar a extração de repositórios para os contextos de educação e saúde.

In [1]:
import requests
import pandas as pd
import time
import logging

As principais palavras chaves usadas por [met]() são análise, portal, publicação, consumir juntamente a dados abertos governamentais ou do governo.

In [20]:
strings_edu = [
    'dados educacao',
    'dados educacao basica',
    'dados educacionais',
    'analise educacao',
    'analise educacao basica',
    'analise educacional',
    'censo educacao superior',
    'dados educacao superior',
    'analise educacao superior',
    'censo profissionais magistério',
    'dados profissionais magistério',
    'analise profissionais magistério',
    'censo escolar',
    'dados escola inep',
    'dados enade',
    'analise enade',
    'dados encceja',
    'analise encceja',
    'dados enem',
    'analise enem',
    'enem por escola',
    'dados prova brasil',
    'analise prova brasil',    
    'dados ideb',
    'indicadores educacionais',
    'dados ies',
    'analise ies',    
    'dados inep',
    'analise inep',
    'microdados inep'
]

In [3]:
strings_saude = [
    'dados sus',
    'dados saúde',
    'dados saude da mulher',
]

Configuração para gerar arquivo de log

In [4]:
logging.basicConfig(level=logging.DEBUG, 
                    filename="log_file.txt", 
                    filemode="a+",
                    format="%(asctime)s - %(levelname)s - %(funcName)s - %(message)s")

logging.info("Extração de dados do Github")

In [5]:
first_search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

Para a acesso a alguns recursos da API do github é preciso se autenticar, como aumentar o limite de requisições. Informações sobre autenticação podem ser encontradas [aqui](https://developer.github.com/v3/#authentication).

In [8]:
credentials = ('<user_name>','<token>')

Limite de requisições sem autenticação

In [9]:
t = requests.get('https://api.github.com/rate_limit')
t.json()

{'resources': {'core': {'limit': 60, 'remaining': 60, 'reset': 1587002412},
  'graphql': {'limit': 0, 'remaining': 0, 'reset': 1587002412},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1587002412},
  'search': {'limit': 10, 'remaining': 10, 'reset': 1586998872}},
 'rate': {'limit': 60, 'remaining': 60, 'reset': 1587002412}}

Limite de requisições com autenticação

In [10]:
t = requests.get('https://api.github.com/rate_limit', auth=credentials)
t.json()

{'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1587002418},
  'search': {'limit': 30, 'remaining': 30, 'reset': 1586998878},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1587002418},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1587002418},
  'source_import': {'limit': 100, 'remaining': 100, 'reset': 1586998878}},
 'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1587002418}}

Verificando limitação de extração de dados da API

In [11]:
page_35 = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(page_35, auth=credentials)
t.json()

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://developer.github.com/v3/search/'}

Informações sobre a ferramenta de pesquisa da API podem ser encontradas [aqui](https://developer.github.com/v3/search/)

In [12]:
url_base = 'https://api.github.com/search/repositories?q='

Podemos adicionar uma ordenação nos resultados, como quantidade de _stars_ de forma descrescente.

In [13]:
sort = '&sort=stars&order=desc'

## Extraindo informações gerais

In [14]:
def extract_results(data):
    
    items_list = []
    
    for item in data.get('items', None):
        
        
        item_dict = {
                'id': item.get('id'),
                'full_name': item.get('full_name', None),
                'description': item.get('description', None),      
                'owner_type': item.get('owner').get('type', None),
                'owner_api_url': item.get('owner').get('url', None),
                'owner_url': item.get('owner').get('html_url', None),
                'api_url': item.get('url', None),
                'url': item.get('html_url', None),
                'fork': item.get('fork', None),
                'created_at': item.get('created_at', None),
                'updated_at': item.get('updated_at', None),
                'pushed_at': item.get('pushed_at', None),
                'size': item.get('size', None),
                'stargazers_count': item.get('stargazers_count', None),
                'language': item.get('language', None),
                'has_issues': item.get('has_issues', None),
                'has_wiki': item.get('has_wiki', None),
                'forks_count': item.get('forks_count', None),
                'forks': item.get('forks', None),
                'open_issues': item.get('open_issues', None),
                'license': item.get('license').get('name', None) if item.get('license', None) else None,
                'timestamp_extract': str(time.time()).split('.')[0]
        }

        items_list.append(item_dict)
            
    return items_list

In [15]:
def scroll_pages(url):
        
    results = requests.get(url, auth=credentials)    
    data = results.json()
    total = data.get('total_count', None)
        
    logging.info("Foram encontrados {0} resultados. Extraindo...".format(total))
        
    items_list = []
    items_list = extract_results(data)
        
    iterations = total // 30 
    
    for iteracao in range(0, iterations):        
        header = results.links
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
                        
            results = requests.get(next_url, auth=credentials)
            data = results.json()
            
            items_list = items_list + extract_results(data)
    
    return items_list

In [23]:
%%time

items_list = []
repositories_df = None

for string in strings_edu:
    url = url_base + string + sort
    
    logging.info("Pesquisando repositórios para a string: '{0}'".format(string))
    
    results = scroll_pages(url)
    items_list = items_list + results
    print('Palavra chave: ', string, ' - Resultados encontrados: ', len(results))
        
repositories_df = pd.DataFrame(items_list)

Palavra chave:  dados educacao  - Resultados encontrados:  61
Palavra chave:  dados educacao basica  - Resultados encontrados:  5
Palavra chave:  dados educacionais  - Resultados encontrados:  28
Palavra chave:  analise educacao  - Resultados encontrados:  52
Palavra chave:  analise educacao basica  - Resultados encontrados:  5
Palavra chave:  analise educacional  - Resultados encontrados:  11
Palavra chave:  censo educacao superior  - Resultados encontrados:  5
Palavra chave:  dados educacao superior  - Resultados encontrados:  5
Palavra chave:  analise educacao superior  - Resultados encontrados:  9
Palavra chave:  censo profissionais magistério  - Resultados encontrados:  0
Palavra chave:  dados profissionais magistério  - Resultados encontrados:  0
Palavra chave:  analise profissionais magistério  - Resultados encontrados:  0
Palavra chave:  censo escolar  - Resultados encontrados:  20
Palavra chave:  dados escola inep  - Resultados encontrados:  9
Palavra chave:  dados enade  - Re

TypeError: 'NoneType' object is not iterable

In [17]:
repositories_df.head(3)

AttributeError: 'NoneType' object has no attribute 'head'

Quantidade de resultados:

In [ ]:
len(repositories_df)

Retirando registros duplicados visto que palavras de busca diferentes podem levar a um mesmo repositório.

In [ ]:
repositories_df = repositories_df.drop_duplicates(['id', 'api_url'])

In [ ]:
len(repositories_df)

In [ ]:
repositories_df.describe()

Quantidade de colunas:

In [ ]:
len(repositories_df.columns)

In [ ]:
repositories_df = repositories_df.sort_values('stargazers_count', ascending=False)

In [ ]:
repositories_df.head(1)

## Extraindo _Commits_, _Contributors_ e dados do _Owner_

In [ ]:
repo_copia = repositories_df

In [ ]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'  
    results = requests.get(commits_url, auth=credentials)
    
    if results.status_code == 409:
        return None
    
    commits = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = results.links


    return commits

In [ ]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    results = requests.get(contributors_url, auth=credentials)
    
    if results.status_code == 204:
        return None
    
    contributors = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = results.links
    
    return contributors

In [ ]:
def extract_owner_data(owner_api_url):
    
    results = requests.get(owner_api_url, auth=credentials)
    data = results.json()

    owner_data = {
        'owner_location': data.get('location', None),
        'owner_email': data.get('email', None),
        'owner_blog': data.get('blog', None),
        'owner_name': data.get('name', None)
    }
    
    return owner_data

In [ ]:
%%time
urls = repositories_df['api_url']

for url in urls:

    owner_api_url = repositories_df.loc[repositories_df["api_url"] == url]['owner_api_url'].item()
    owner_data = extract_owner_data(owner_api_url)
    commits = extract_commits(url)
    contributors = extract_contributors(url)
    
    logging.info("Repositório: {0}".format(url))
    logging.info("Tem {0} Commits - {1} Contributors".format(commits,contributors))
    logging.info("Owner location: {0}".format(owner_data.get('owner_location')))

    repositories_df.loc[repositories_df["api_url"] == url, 'commits'] = commits
    repositories_df.loc[repositories_df["api_url"] == url, 'contributors'] = contributors
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_location'] = owner_data.get('owner_location')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_email'] = owner_data.get('owner_email')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_blog'] = owner_data.get('owner_blog')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_name'] = owner_data.get('owner_name')

Agora devemos ter mais 6 colunas

In [ ]:
len(repositories_df.columns)

In [ ]:
repositories_df.head(3)

Conferindo valores nulos

In [ ]:
len(repositories_df.loc[repositories_df['commits'].isnull()][['api_url', 'commits', 'contributors']])

Alguns repositórios realmente não tem nenhum commit como o [ccdpoa/ocida](https://github.com/ccdpoa/ocida).

In [ ]:
repositories_df.loc[repositories_df['contributors'].isnull()][['id', 'url', 'api_url', 'commits', 'contributors']]

In [ ]:
repositories_df = repositories_df.loc[repositories_df['contributors'].notnull()]
len(repositories_df)

Salvando os repositórios

In [ ]:
repositories_df.to_csv('../data/repositories_edu.csv', index=False)

## Extraindo contribuidores dos repositórios

In [ ]:
def get_contributors(data, repo_data):

    list_contributors = []

    for item in data:        
        contributor = {
            'repo_id': repo_data.get('repo_id', None),
            'repo_name': repo_data.get('repo_name', None),
            'repo_url': repo_data.get('repo_url', None),
            'repo_api_url': repo_data.get('repo_api_url', None),
            'contributor_id': item.get('id', None),
            'contributor_login': item.get('login', None),
            'contributor_type': item.get('type', None),
            'contributor_url': item.get('html_url', None),
            'contributor_api_url': item.get('url', None),
            'timestamp_extract': str(time.time()).split('.')[0]
        }

        list_contributors.append(contributor)

    return list_contributors

In [ ]:
def scroll_contributors(url, repo_data):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    
    if results.status_code is 204:
        return None
    
    data = results.json()
    list_contributors = get_contributors(data, repo_data)
    header = results.links
    
    while header.get('next', False):
        
        next_url = header.get('next').get('url')            
        results = requests.get(next_url, auth=credentials)
        data = results.json()
        list_contributors = list_contributors + get_contributors(data, repo_data)  
        header = results.links
        
    return list_contributors

In [ ]:
def search_contributors(repositories_df):
    
    urls = repositories_df['api_url']
    list_contributors_all_repo = []
    
    for url in urls:
        logging.info('Extraindo contribuidores de: {0}'.format(url))
        
        repo_data = {
                'repo_id': repositories_df.loc[repositories_df["api_url"] == url, 'id'].values[0],
                'repo_name': repositories_df.loc[repositories_df["api_url"] == url, 'full_name'].values[0],
                'repo_url': repositories_df.loc[repositories_df["api_url"] == url, 'url'].values[0],
                'repo_api_url': url,
            }
        
        url_contributors = url + '/contributors'        
        contributors = scroll_contributors(url_contributors, repo_data)
        
        if contributors:
            list_contributors_all_repo = list_contributors_all_repo + contributors
    
    contributors_df = pd.DataFrame(list_contributors_all_repo)     
        
    return contributors_df

In [ ]:
%%time
contributors_df = search_contributors(repositories_df)

In [ ]:
contributors_df.head(3)

Verificando se há contribuidores repetidos para um mesmo repositório.

In [ ]:
contributors_df[contributors_df.duplicated(['contributor_id', 'repo_id'])]

Salvando dataframe com mapeamento de repositórios e contribuidores.

In [ ]:
contributors_df.to_csv('../data/contributors_edu.csv', index=False)